<a href="https://colab.research.google.com/github/gabrimatx/Jupyter-Notebooks/blob/main/CUDA_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA Notebook

In [1]:
!nvcc --version
!nvidia-smi
!nvidia-smi --query-gpu=compute_cap --format=csv
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Tue Jan 30 23:00:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  7

## CUDA: Compute Unified Device Architecture
- Enables explicit GPU memory management.
- GPU is viewed as *compute device* and is a co-processor to the CPU, has its own **DRAM** and runs many threads in parallel.
---
- The GPU architecture is built around a scalable array of streaming multiprocessors (SM).
- Every SM in a GPU support the concurrent execution of hundreds of threads.
- Every SM executes thread in groups of 32 called **warps**. All threads in a warp execute the same instruction at the same time.

# Typical structure of a CUDA program
- GPU memory allocation.
- Transfer data to CPU.
- Execute CUDA kernel.
- Copy results from GPU's memory.
- Reinitialize device.

# Hello world

In [2]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 1, thread: 0
Hello from block: 1, thread: 1



# Function decorations
`__device__` : a function that can only be called from within a kernel, i.e. not from the host.  

`__host__` : a function that can only run on the host. The __host__ qualifier is typically omitted, unless used in combination with __device__ to indicate that the function can run on both the host and the device. Such a scenario implies the generation of two compiled codes for the function.

`__global__` : a function that can be called from the host and executed on the device. In CC 3.5 and above, the device can also call __global__ functions, using a feature called dynamic parallelism.

# CUDA Memory Management Functions

- `cudaMalloc(address, size)`: allocates object in device global memory.
- `cudaFree(address)`: frees object from device global memory.
- `cudaMemcpy(destination, source, bytes_copied, direction_of_transfer)`: memory data transfer, synchronous.

# Vector addition

In [3]:
%%cuda
#include <stdio.h>
#include <cuda.h>
__global__ void vecAddKernel(float *A_d, float *B_d, float *C_d, int n){
    int i = threadIdx.x + blockDim.x * blockIdx.x;
    if (i < n) C_d[i] = A_d[i] + B_d[i];
}

void vecAdd(float *A_h, float *B_h, float *C_h, int n){
    int size  = n * sizeof(float);
    float *A_d, *B_d, *C_d;




    // Transfer A and B to device memory
    cudaMalloc(&A_d, size);
    cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice);
    cudaMalloc(&B_d, size);
    cudaMemcpy(B_d, B_h, size, cudaMemcpyHostToDevice);

    // Allocate device memory for result
    cudaMalloc(&C_d, size);

    // Kernel invocation
    vecAddKernel<<<ceil(n/256.0),256>>>(A_d, B_d, C_d, n);
    cudaDeviceSynchronize();



    // Transfer C to host
    cudaMemcpy(C_h, C_d, size, cudaMemcpyDeviceToHost);
    printf("C = [%f %f %f]", C_h[0], C_h[1], C_h[2]);

    // Free device memory
    cudaFree(A_d); cudaFree(B_d); cudaFree(C_d);

}

int main(){
    float A[] = {1,2,3}, B[] = {1,2,3}, C[3];
    vecAdd(A, B, C, 3);
    return 0;
}

C = [2.000000 4.000000 6.000000]


# Kernel restrictions
- Can only access device memory.
- Must be of void type.
- Does not support a variable number of arguments.
- Does not support static variables.
- Does not support pointers to functions.
- Is async w.r.t. the host.

# Intrinsic variables
- `blockDim`: Contains the size of each block e.g. $(B_x, B_y, B_z)$.
- `gridDim`: Contains the size of the grid in blocks e.g. $(G_x, G_y, G_z)$.
- `threadIdx`: The $(x,y,z)$ position of the thread within a block, with $x\in [0, B_x-1],y\in [0,B_y - 1], z\in [0, B_z-1]$.
- `blockIdx`: The $(b_x,b_y,b_z)$ position of thread's block within the grid, with $b_x\in [0, G_x-1],b_y\in [0,G_y - 1], b_z\in [0, G_z-1]$.

`blockDim` and `gridDim` are of type `dim3` a type of integer vectors based on `uint3`. When a variable of type `dim3` is initialized, non defined components are set to 1. Components in a `dim3` variable are accessible with `x,y,z`.

To compute the unique Id of a thread, that can be considered an element of a 6-D array
`Thread t[gridDim.z][gridDim.y][gridDim.x][blockDim.z][blockDim.y][blockDim.x];`
we can use the formula:


```
int myID = (blockIdx.z * gridDim.x * gridDim.y + blockIdx.y * gridDim.x + blockIdx.x) * blockDim.x * blockDim.y * blockDim.z + threadIdx.z * blockDim.x * blockDim.y + threadIdx.y * blockDim.x + threadIdx.x;
```




# Hello word #2

In [4]:
%%cuda
#include <stdio.h>
__global__ void hello(){
    int myID = (blockIdx.z * gridDim.x * gridDim.y + blockIdx.y * gridDim.x + blockIdx.x) * blockDim.x * blockDim.y * blockDim.z + threadIdx.z * blockDim.x * blockDim.y + threadIdx.y * blockDim.x + threadIdx.x;
    printf("hello from %d\n", myID);
}

int main(){
        hello<<<2, 4>>>();
        cudaDeviceSynchronize();
}

hello from 0
hello from 1
hello from 2
hello from 3
hello from 4
hello from 5
hello from 6
hello from 7



# Cooperation withing same block
The programmer can partition the problem into coarse sub-problems that can be solved independently in parallel by blocks of threads, and each sub-problem into finer pieces that can be solved cooperatively in parallel by all threads within the block.

- For all purposes, we can only assume that a thread terminates when the block it belongs to terminates.
- When an SM completes a block, it switches to another one.
- A switch is also possible if the threads of a block stall while waiting. Having other resident blocks depends on resource availability (e.g. shared memory, registers).
- **Important**: The existence of multiple warp schedulers in SMs means that at any point in time multiple warps from the same block may be executing.

# Device properties
`maxThreadsPerBlock`: the maximal number of threads allowed in a block. Some devices allow up to 1024 threads in each block and other devices allow fewer.

`maxThreadsPerMultiProcessor`: the maximum resident threads per multiprocessor. This can limit the numbers of concurrent blocks.

`maxBlocksPerMultiProcessor`: maximum number of resident blocks per multiprocessor.

`multiProcessorCount`: the number of SM.


# Matrix multiplication

In [6]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define M 8
#define P 4
#define N 2
#define BLOCK_SIZE 16


__global__ void mat_mult_kernel(double* A_d, double* B_d, double* C_d, int m, int p, int n) {
    int i = blockDim.y * blockIdx.y + threadIdx.y;
    int j = blockDim.x * blockIdx.x + threadIdx.x;

    double sum = 0.0;
    if (i < m && j < n) {
      for (int k = 0; k < p; k++) {
          sum += A_d[i * p + k] * B_d[k * n + j];
      }
      C_d[i * n + j] = sum;
    }
}

void print_matrix(char *name, int rows, int cols, double *A)
{
    printf("%s:\n", name);
    for (int i = 0; i < rows; i++)
    {
        for (int j = 0; j < cols; j++)
        {
            printf("%.2f\t", A[i * cols + j]);
        }
        printf("\n");
    }
    printf("\n");
}


int main() {
    double A_h[M * P];
    double B_h[P * N];
    double C_h[M * N];
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < P; j++) {
            A_h[i * P + j] = i;
        }
    }

    for (int i = 0; i < P; i++) {
        for (int j = 0; j < N; j++) {
            B_h[i * N + j] = i;
        }
    }

    double *A_d, *B_d, *C_d;
    cudaMalloc(&A_d, M * P * sizeof(double));
    cudaMalloc(&B_d, P * N * sizeof(double));
    cudaMalloc(&C_d, M * N * sizeof(double));
    cudaMemcpy(A_d, A_h, M * P * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(B_d, B_h, P * N * sizeof(double), cudaMemcpyHostToDevice);
    dim3 b(BLOCK_SIZE, BLOCK_SIZE);
    dim3 g(ceil(N/(double) b.x), ceil(M/(double) b.y));
    mat_mult_kernel<<<g, b>>>(A_d, B_d, C_d, M, P, N);
    cudaMemcpy(C_h, C_d, M * N * sizeof(double), cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();

    cudaFree(A_d); cudaFree(B_d); cudaFree(C_d);

    print_matrix("A", M, P, A_h);
    print_matrix("B", P, N, B_h);
    print_matrix("C", M, N, C_h);
}


A:
0.00	0.00	0.00	0.00	
1.00	1.00	1.00	1.00	
2.00	2.00	2.00	2.00	
3.00	3.00	3.00	3.00	
4.00	4.00	4.00	4.00	
5.00	5.00	5.00	5.00	
6.00	6.00	6.00	6.00	
7.00	7.00	7.00	7.00	

B:
0.00	0.00	
1.00	1.00	
2.00	2.00	
3.00	3.00	

C:
0.00	0.00	
6.00	6.00	
12.00	12.00	
18.00	18.00	
24.00	24.00	
30.00	30.00	
36.00	36.00	
42.00	42.00	




# Matrix multiplication with shared memory

In [7]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define M 8
#define P 4
#define N 2
#define BLOCK_SIZE 2

void print_matrix(char *name, int rows, int cols, double *A)
{
    printf("%s:\n", name);
    for (int i = 0; i < rows; i++)
    {
        for (int j = 0; j < cols; j++)
        {
            printf("%.2f\t", A[i * cols + j]);
        }
        printf("\n");
    }
    printf("\n");
}

__global__ void mat_mult_kernel(double* A_d, double* B_d, double* C_d, int m, int p, int n) {
    int tx = threadIdx.x; int ty = threadIdx.y;
    int i = blockDim.y * blockIdx.y + ty;
    int j = blockDim.x * blockIdx.x + tx;

    __shared__ double A_ds[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ double B_ds[BLOCK_SIZE][BLOCK_SIZE];

    double sum = 0.0;
    for (int t = 0; t < ceil(p / (double) BLOCK_SIZE); t++) {
        if (i < m && (t * BLOCK_SIZE + tx) < p) A_ds[ty][tx] = A_d[i * p + t * BLOCK_SIZE + tx]; // Go to the line, go to the tile, go to the thread
        if ((t * BLOCK_SIZE + ty) < p && j < n) B_ds[ty][tx] = B_d[(t * BLOCK_SIZE + ty) * n + j]; // Select tile, select thread, go to column
        __syncthreads();

        for (int k = 0; k < BLOCK_SIZE; k++) sum += A_ds[ty][k] * B_ds[k][tx]; // Each thread performs row column mult of shared memories
        __syncthreads();
    }
    if (i < m && j < n) C_d[i * n + j] = sum; // Put sum into output matrices
}

int main() {
    double A_h[M * P];
    double B_h[P * N];
    double C_h[M * N];
    for (int i = 0; i < M; i++) {
        for (int j = 0; j < P; j++) {
            A_h[i * P + j] = i;
        }
    }

    for (int i = 0; i < P; i++) {
        for (int j = 0; j < N; j++) {
            B_h[i * N + j] = i;
        }
    }

    double *A_d, *B_d, *C_d;
    cudaMalloc(&A_d, M * P * sizeof(double));
    cudaMalloc(&B_d, P * N * sizeof(double));
    cudaMalloc(&C_d, M * N * sizeof(double));
    cudaMemcpy(A_d, A_h, M * P * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(B_d, B_h, P * N * sizeof(double), cudaMemcpyHostToDevice);
    dim3 b(BLOCK_SIZE, BLOCK_SIZE);
    dim3 g(ceil(N/(double) b.x), ceil(M/(double) b.y));
    mat_mult_kernel<<<g, b>>>(A_d, B_d, C_d, M, P, N);
    cudaMemcpy(C_h, C_d, M * N * sizeof(double), cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();

    cudaFree(A_d); cudaFree(B_d); cudaFree(C_d);

    print_matrix("A", M, P, A_h);
    print_matrix("B", P, N, B_h);
    print_matrix("C", M, N, C_h);
}


A:
0.00	0.00	0.00	0.00	
1.00	1.00	1.00	1.00	
2.00	2.00	2.00	2.00	
3.00	3.00	3.00	3.00	
4.00	4.00	4.00	4.00	
5.00	5.00	5.00	5.00	
6.00	6.00	6.00	6.00	
7.00	7.00	7.00	7.00	

B:
0.00	0.00	
1.00	1.00	
2.00	2.00	
3.00	3.00	

C:
0.00	0.00	
6.00	6.00	
12.00	12.00	
18.00	18.00	
24.00	24.00	
30.00	30.00	
36.00	36.00	
42.00	42.00	


